## AOS 575, Problem Set 4

### Imports

In [228]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd # Only used for data statistics during debugging

In [229]:
%config InlineBackend.figure_format = 'png'

## Problem 1

#### Definitions

Function for $u$ (horizontal component of velocity)

In [230]:
def u_x(x, y, t):
    ''' Define horizontal component of velocity. '''
    return (np.sin(np.pi*x)**2)*np.sin(2*np.pi*y)*np.cos(np.pi*t/5)

Function for $v$ (vertical component of velocity)

In [231]:
def u_y(x, y, t):
    ''' Define vertical component of velocity. '''
    return -np.sin(2*np.pi*x)*(np.sin(np.pi*y)**2)*np.cos(np.pi*t/5)

Function to select $r$ value at each grid point

In [232]:
def r_sel(c, v, print_bool=False):
    ''' Decide r value to use at each grid point given a constant and variable array. '''
    
    # Lengths in the horizontal (x) and vertical (y) directions
    l_x, l_y = c.shape[0], c.shape[1]
    # Initialize array for r
    r = np.full(shape=(l_x, l_y), fill_value=np.nan)
    # Iterate over each grid point
    for i in range(0, l_x):
        for j in range(0, l_y):
            # Print values
            print('x: {0:4d} | y: {1:4d} | C: {2:5.2f}, V:{3:5.2f}'.format(i, j, c[i, j], v[i, j])) if print_bool else None
            # Select lesser of both values
            r[i, j] = np.nanmin([c[i, j], v[i, j]])

    return r

Update staggered grid values based on current timestep

In [233]:
def regen(phi, X_stag, Y_stag):
    ''' Regenerate staggered grid values for fields that are functions of the velocity (rho, phi). '''
    
    # Define number of points in x and y, assuming same number of points in each direction
    ny, nx = phi.shape
    
    # Initialize fresh staggered grids
    phi_stag = np.full(shape=(ny-1, nx-1), fill_value=np.nan, dtype=float)
    
    # Iterate over points and average from surrounding points (average of 4 'surrounding' points)
    for i in range(1, ny):
        for j in range(1, nx):
            phi_stag[i-1, j-1] = np.mean([phi[i, j], phi[i, j-1], phi[i-1, j], phi[i-1, j-1]])
        
    return phi_stag

#### Model initialization
Initialize parameters for the model, grid, and data structures.

In [398]:
''' Define grid parameters. '''

x_min, x_max = [0, 1] # Horizontal domain limits
y_min, y_max = [0, 1] # Vertical domain limits
dx = 0.02 # Horizontal grid spacing
dy = dx # Vertical grid spacing

# Generate horizonal and vertical grid points
gp_x = np.arange(x_min, x_max+dx, dx) 
gp_y = np.arange(y_min, y_max+dy, dy) 
X, Y = np.meshgrid(gp_x, gp_y)

# Generate 1/2-point staggered grid
gp_x_stag = np.arange(x_min+(dx/2), x_max, dx) 
gp_y_stag = np.arange(x_min+(dx/2), x_max, dx) 
X_stag, Y_stag = np.meshgrid(gp_x_stag, gp_y_stag)

''' Define model parameters. '''

c = 0.5 # Courant number
dt = 0.5*dx # Note: maximum value of velocity field is 1
t_max = 7.5 # Maximum time
times = np.arange(0, t_max, dt) # Define time array

''' Initialize velocity field, tracer concentration, and density arrays. '''
# Note: the density array is included for Easter's pseudocompressibility method. As it is a 'pseudo-density', it will be set to be 1.

# Base grid: initiate 3-dimensional arrays dictated by (x, y, t)
u = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=np.nan)
v = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=np.nan)
phi = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=np.nan)
rho = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=1)
F = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=np.nan)
G = np.full(shape=(len(gp_x), len(gp_y), len(times)), fill_value=np.nan)

# Staggered grid: initiate 3-dimensional arrays dictated by (x, y, t)
u_stag = np.full(shape=(len(gp_x_stag), len(gp_y_stag), len(times)), fill_value=np.nan)
v_stag = np.full(shape=(len(gp_x_stag), len(gp_y_stag), len(times)), fill_value=np.nan)
phi_stag = np.full(shape=(len(gp_x_stag), len(gp_y_stag), len(times)), fill_value=np.nan)
rho_stag = np.full(shape=(len(gp_x_stag), len(gp_y_stag), len(times)), fill_value=1)

# Starting position of tracer concentration
x0, y0 = 1/4, 1/4

# Initialize tracer concentrations
r_const = np.full(shape=(len(gp_x), len(gp_y)), fill_value=1) # Constant argument for r
r_var = 4*np.sqrt((X-x0)**2 + (Y-y0)**2) # Variable argument for r
r = r_sel(r_const, r_var, print_bool=print_bool)

r_const_stag = np.full(shape=(len(gp_x_stag), len(gp_y_stag)), fill_value=1) # Constant argument for r
r_var_stag = 4*np.sqrt((X_stag-x0)**2 + (Y_stag-y0)**2) # Variable argument for r
r_stag = r_sel(r_const_stag, r_var_stag, print_bool=print_bool)

#### Implentation of numerical methods

__Piecewise linear method__: use second-to-last form from Lecture 9: Tracer Advection, Slide 10:

$ F_{i+1/2} = \frac{c \Delta x}{\Delta t} \left[\phi_i + \frac{1}{2} \Delta \phi_i (1-c) \right] $
* Note: assume that $\Delta \phi$ indicates change in value between grid points.

In [236]:
def plm(phi, phi_stag, u, v, c, dx, dy, dt):
    ''' 
    Piecewise linear method, generates F_{i(+/-)1/2} and G_{i(+/-)1/2} for each grid point. 
    Work off of the base grid (X, Y).
    '''

    # Redefine grid size
    ny, nx = phi.shape
    
    # Initialize arrays for F and G. Initialize at zero to handle domain boundaries.
    F = np.full(shape=phi.shape, fill_value=0, dtype=float)
    G = np.full(shape=phi.shape, fill_value=0, dtype=float)
    
    # Iterate over the arrays
    for i in range(1, ny-2): # iterate over rows
        for j in range(1, nx-2): # iterate over columns
            
            sigma_x = ((phi[i, j+1] - phi[i, j])/dx)
            sigma_y = ((phi[i+1, j] - phi[i, j])/dy)
            
            F[i, j] = u[i, j]*(phi[i, j] + (1-(u[i, j]*dt/dx))*dx*sigma_x/2) # 5.55
            G[i, j] = v[i, j]*(phi[i, j] + (1-(v[i, j]*dt/dx))*dy*sigma_y/2)
            
            # Check for TVD
            if F[i, j] + G[i, j] >= 1:
                print('\t TVD violated at ({0}, {1}): {2:.3f}'.format(i, j, F[i, j] + G[i, j])) if print_bool else None
                      
    return F, G

__Pseudocompressibility method__: Lecture 9: Tracer Advection, Slide 22.

In [412]:
def ps(base_fields, stag_fields, t, c, dx, dy, dt, X_stag, Y_stag):
    ''' 
    Easter's pseudocompressibility method. 
    '''
    
    # Unpack field arrays
    # Note: For staggered grids, trailing by one grid cell: i+(1/2) --> i, i-(1/2) --> i-1
    u, v, rho, phi = base_fields
    u_stag, v_stag, rho_stag, phi_stag = stag_fields
    
    # Redefine grid size
    ny, nx = u.shape
    
    # Initialize intermediate arrays - the '_n' suffix indicates n+1 timestep
    rho_s = np.full(shape=u.shape, fill_value=rho, dtype=float)
    rho_phi_s = np.full(shape=u.shape, fill_value=(rho*phi), dtype=float)
    phi_s = np.full(shape=u.shape, fill_value=phi, dtype=float)
    rho_n = np.full(shape=u.shape, fill_value=rho, dtype=float)
    rho_phi_n = np.full(shape=u.shape, fill_value=(rho*phi), dtype=float)
    phi_n = np.full(shape=u.shape, fill_value=phi, dtype=float)
    
    # Enable to prevent Strang splitting
    # t = 0
    # Enable to disable pseudocompressibility
    ps_null = True
    
    if t % 2 == 0:
        F, _ = plm(phi, phi_stag, u_stag, v_stag, c, dx, dy, dt)

        for i in range(1, ny-1): 
            for j in range(1, nx-1): 
                rho_phi_s[i, j] = rho[i, j]*phi[i, j] + (dt/dx)*(F[i, j-1] - F[i, j])
                rho_s[i, j] = rho[i, j] + (dt/dx)*(u_stag[i, j-1] - u_stag[i, j])
                # Nullify pseudocompressibility if boolean tripped
                rho_s[i, j] = 1 if ps_null else rho_s[i, j]
                phi_s[i, j] = rho_phi_s[i, j]/rho_s[i, j]

        phi_stag = regen(phi_s, X_stag, Y_stag)
        _, G = plm(phi_s, phi_stag, u_stag, v_stag, c, dx, dy, dt)

        for i in range(1, ny-1): 
            for j in range(1, nx-1): 
                rho_phi_n[i, j] = rho_phi_s[i, j] + (dt/dy)*(G[i-1, j] - G[i, j])
                rho_n[i, j] = rho_s[i, j] + (dt/dy)*(v_stag[i-1, j] - v_stag[i, j])
                phi_n[i, j] = rho_phi_n[i, j]/rho_n[i, j]
    else:
        
        _, G = plm(phi, phi_stag, u_stag, v_stag, c, dx, dy, dt)

        for i in range(1, nx-1): 
            for j in range(1, ny-1): 
                rho_phi_s[i, j] = rho[i, j]*phi[i, j] + (dt/dy)*(G[i-1, j] - G[i, j])
                rho_s[i, j] = rho[i, j] + (dt/dy)*(v_stag[i-1, j] - v_stag[i, j])
                # Nullify pseudocompressibility if boolean tripped
                rho_s[i, j] = 1 if ps_null else rho_s[i, j]
                phi_s[i, j] = rho_phi_s[i, j]/rho_s[i, j]

        phi_stag = regen(phi_s, X_stag, Y_stag)
        F, _ = plm(phi_s, phi_stag, u_stag, v_stag, c, dx, dy, dt)

        for i in range(1, nx-1): 
            for j in range(1, ny-1): 
                rho_phi_n[i, j] = rho_phi_s[i, j] + (dt/dx)*(F[i, j-1] - F[i, j])
                rho_n[i, j] = rho_s[i, j] + (dt/dx)*(u_stag[i, j-1] - u_stag[i, j])
                phi_n[i, j] = rho_phi_n[i, j]/rho_n[i, j]
        
    return F, G, rho_s, phi_n

In [9]:
# Switch to turn printing on or off for debugging
print_bool = False

#### Run the numerical scheme

In [413]:
# Define artificial loop step limit for debugging
loop_max = 500

rho_s = rho.copy()

for i, t in enumerate(times):
    # Cut at initial timestep for now
    if i > loop_max:
        break
        
    # Print variables for each time step
    print('Time step: {0:4d} | Time: {1:4.2f} s'.format(i, t)) if print_bool else None
    
    # Assign velocity field and tracer concentration for the given timestep
    u[:, :, i], v[:, :, i] = u_x(X, Y, t), u_y(X, Y, t)
    u_stag[:, :, i], v_stag[:, :, i] = u_x(X_stag, Y_stag, t), u_y(X_stag, Y_stag, t)
     
    # Initialize non-velocity fields
    if i == 0:
        phi[:, :, i] = (1/2)*(1 + np.cos(np.pi*r)) 
        phi_stag[:, :, i] = (1/2)*(1 + np.cos(np.pi*r_stag))
    
    base_fields = [u[:, :, i], v[:, :, i], rho[:, :, i], phi[:, :, i]]
    stag_fields = [u_stag[:, :, i], v_stag[:, :, i], rho_stag[:, :, i], phi_stag[:, :, i]]
    F[:, :, i+1], G[:, :, i+1], rho_s[:, :, i+1], phi[:, :, i+1] = ps(base_fields, stag_fields, i, c, dx, dy, dt, X_stag, Y_stag)
    phi_stag[:, :, i+1] = regen(phi[:, :, i+1], X_stag, Y_stag)
    
    print ('-----------------') if print_bool else None

#### Visualization

Plot data

In [427]:
def plot(x, y, dx, u, v, phi, F, G, rho, times, t=0):
    
    # Select timesteps of interest
    u, v, phi, phi_ref = u[:, :, t], v[:, :, t], phi[:, :, t], phi[:, :, 0]
    F, G = F[:, :, t], G[:, :, t]
    rho = rho[:, :, t]
    
    fig, axs = plt.subplots(figsize=(10, 5), ncols=4, sharey=True)
    [ax_phi, ax_F, ax_G, ax_rho] = axs
    
    ''' Quiver plot (q) for velocity & tracer field. '''
    q = ax_phi.quiver(x, y, u, v)   
    dc = 0.1
    P_ref = ax_phi.contour(x, y, phi_ref, levels=np.arange(dc, 1+dc, dc), colors='k', alpha=0.25)
    P = ax_phi.contour(x, y, phi, levels=np.arange(dc, 1+dc, dc))
    
    ''' Contour plot for fluxes. '''    
    norm_flux = matplotlib.colors.TwoSlopeNorm(vcenter=0)
    f = ax_F.contourf(x, y, F, cmap='RdBu_r', levels=16, norm=norm_flux)
    g = ax_G.contourf(x, y, G, cmap='RdBu_r', levels=16, norm=norm_flux)
    
    ''' Contour plot for density. '''
    p = ax_rho.contourf(x, y, rho, cmap='plasma')
    
    ''' Figure formatting. '''
    # Equal aspect ratio everywhere
    [ax.set_aspect('equal') for ax in axs]
    
    def colorbar(ax, field, plot_var, name):
        cax = fig.add_axes([ax.get_position().x0,
                            ax.get_position().y0 - 0.1,
                            ax.get_position().width,
                            0.02])
        cbar = fig.colorbar(plot_var, cax=cax, orientation='horizontal')
        cbar.set_label(name, labelpad=15)
        cbar.ax.tick_params(rotation=90)
        
        return cax, cbar
    
    # Format colorbars
    cax_phi, cbar_phi = colorbar(ax_phi, phi, P, '$\psi$')
    cax_F, cbar_F = colorbar(ax_F, F, f, 'F')
    cax_G, cbar_G = colorbar(ax_G, G, g, 'G')
    cax_rho, cbar_rho = colorbar(ax_rho, rho, p, '$p$')
    
    # Figure formatting
    fs_ax = 10
    
    ax_phi.set_title('Tracer concentration ($\psi$)', fontsize=fs_ax)
    ax_F.set_title('Horizontal flux (F)', fontsize=fs_ax)
    ax_G.set_title('Vertical flux (G)', fontsize=fs_ax)
    ax_rho.set_title('Pseudodensity (h)', fontsize=fs_ax)
    
    fig.suptitle('Time: {0:.2f} s'.format(times[t]), y=0.8)
    # plt.show()
    
    plt.savefig('figs/output-dx_{0:.2f}-t{1}_nopseudo.png'.format(dx, t), dpi=300, bbox_inches='tight')

In [430]:
plot(X, Y, dx, u, v, phi, F, G, rho_s, times, t=500)

Animation

In [ ]:
from matplotlib import animation
import matplotlib.pyplot as plt_
plt_.rcParams["animation.html"] = "jshtml"
plt_.ioff()
plt_.cla()

# Set up plotting
fig, ax = plt_.subplots(figsize=(4, 4))

# Animation function
def animate(i):
    # Multiplier to skip over frames
    n = 10
    # Clear past iteration's data
    ax.cla()
    # Quiver plot
    q = ax.quiver(X, Y, u[:, :, n*i], v[:, :, n*i])
    # Contour plot
    P = ax.contour(X, Y, phi[:, :, n*i], levels=np.arange(0.1, 1, 0.1))
    # Figure formatting
    ax.set_title('Time: {0:.2f} s'.format(times[n*i]))
    ax.set_aspect('equal')

# Animation generation and save settings
intv = 75 
fps = 1000/intv
animation.FuncAnimation(fig, animate, frames=50, interval=intv)
# anim = animation.FuncAnimation(fig, animate, frames=10, interval=intv)
# anim.save('figs/animation_20221108-success-dx_0.02.gif', dpi=300)

Run this cell to allow for single plots again

In [2086]:
# Reset rcParams
plt.rcParams.update(plt.rcParamsDefault)
plt.gca()

<AxesSubplot:title={'center':'Time: 4.95 s'}>

Function to plot grid

In [1088]:
def grid_plot(x, y, x_int, y_int,bounds=None):
    
    fig, ax = plt.subplots(figsize=(4,4))
    im_grid = ax.scatter(x, y, s=1, c='k', alpha=0.5, label='Base')
    im_grid_int = ax.scatter(x_int, y_int, s=0.5, c='tab:blue', alpha=0.5, label='Intermediate')
    
    for i in range(0, len(y)):
        for j in range(0, len(x)):
            if i % 10 == 0 and j % 10 == 0:
                ax.text(x[i, j], y[i, j], '({0}, {1})'.format(i, j))
    
    if bounds:
        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])
    
    ax.set_aspect('equal')
    fig.legend(frameon=False, bbox_to_anchor=(1.275, 0.9))
    plt.show()

#### Save model runs for comparison

dx = dy = 0.02

In [380]:
# Strang splitting used
phi_split_dx002 = phi.copy()
F_split_dx002 = F.copy()
G_split_dx002 = G.copy()

In [414]:
# Strang splitting, no pseudocompressibility
phi_split_dx002_nopseudo = phi.copy()
F_split_dx002_nopseudo = F.copy()
G_split_dx002_nopseudo = G.copy()

In [383]:
# Strang splitting not used
phi_unsplit_dx002 = phi.copy()
F_unsplit_dx002 = F.copy()
G_unsplit_dx002 = G.copy()

dx = dy = 0.01

In [387]:
# Strang splitting used
phi_split_dx001 = phi.copy()
F_split_dx001 = F.copy()
G_split_dx001 = G.copy()

In [390]:
# Strang splitting not used
phi_unsplit_dx001 = phi.copy()
F_unsplit_dx001 = F.copy()
G_unsplit_dx001 = G.copy()

Plot differences between split and unsplit methods

In [436]:
eval_step = 500
dx_split = 0.02
dphi = phi_split_dx002[:, :, eval_step]-phi_split_dx002_nopseudo[:, :, eval_step]
fig, ax = plt.subplots(figsize=(3, 3))

norm = matplotlib.colors.TwoSlopeNorm(vcenter=0)

im = ax.contourf(X, Y, dphi, levels=16, cmap='RdBu_r', norm=norm)

cax = fig.add_axes([ax.get_position().x1 + 0.02,
                            ax.get_position().y0,
                            0.03,
                            ax.get_position().height])
cbar = fig.colorbar(im, cax=cax)
cbar.set_label('$\Delta \psi$', labelpad=15, rotation=270)

ax.set_title('$\Delta$ x= 0.02, pseudo - no_pseudo')
ax.set_aspect('equal')

# plt.show()
plt.savefig('figs/nopseudo-t{0}-dx_{1:.2f}.png'.format(eval_step, dx_split), dpi=300, bbox_inches='tight')

## Problem 3

In [555]:
R = 1

dx0, dy0, dx1, dy1 = 1, 1, 1/np.sqrt(2), 1/np.sqrt(2)
rx0, ry0 = R**2/dx0, R**2/dy0
rx1, ry1 = R**2/dx1, R**2/dy1

N = 10

kdp = np.linspace(0, 1, N)
ldp = np.linspace(0, 1, N)

sk, ck = np.sin(kdp), np.cos(kdp)
sl, cl = np.sin(ldp), np.cos(ldp)

In [561]:
K0, L0 = np.meshgrid(kdp, ldp)
Z0 = 1 + rx0**2*np.sin(K0)**2*np.cos(L0)**2 + ry0**2*np.sin(L0)**2*np.cos(K0)**2

K1, L1 = np.meshgrid(kdp, ldp)
Z1 = 1 + rx1**2*np.sin(K1)**2*np.cos(L1)**2 + ry1**2*np.sin(L1)**2*np.cos(K1)**2

fig, axs = plt.subplots(figsize=(8, 4), ncols=2, subplot_kw={"projection": "3d"})

axs[0].plot_surface(K0, L0, np.sqrt(Z0), cmap='coolwarm', antialiased=False)
axs[0].set_title('wavevector aligned')
axs[1].plot_surface(K1, L1, np.sqrt(Z1), cmap='coolwarm', antialiased=False)
axs[1].set_title('wavevector 45 deg offset')

for ax in axs:
    ax.set_xlabel('$s_k$', fontsize=12)
    ax.set_ylabel('$s_l$', fontsize=12)
    ax.set_zlabel('$\omega$ / f', fontsize=12)

# plt.show()
plt.savefig('figs/dispersion.png', dpi=300, bbox_inches='tight')